In [1]:
# path = 'hdfs://orion11:14001/reddit/2012/RC_2012-01.bz2'
# path = 'hdfs://orion11:14001/reddit/2017/RC_2017-01.bz2'
path = 'hdfs://orion11:14001/reddit/sampled_reddit'
# path_all = 'hdfs://orion11:14001/reddit/*/*.bz2'

In [2]:
df = spark.read.json(path)
# df = spark.read.json(path_all)
df.head()

Row(archived=True, author='halftone84', author_cakeday=None, author_flair_css_class=None, author_flair_text=None, body='I read the title and thought of that cheating bitch clown from the glassjaw video', body_html=None, controversiality=0, created=None, created_utc='1309478410', distinguished=None, downs=0, edited='false', gilded=0, id='c22x4bc', link_id='t3_idgji', mod_reports=None, name='t1_c22x4bc', parent_id='t3_idgji', removal_reason=None, replies=None, retrieved_on=1427302517, saved=None, score=1, score_hidden=False, stickied=None, subreddit='AskReddit', subreddit_id='t5_2qh1i', ups=1, user_reports=None)

In [3]:
df.createOrReplaceTempView('reddit_data')

In [4]:
# query = '''
# SELECT 
#     subreddit_id
# FROM
#    reddit_data
# '''

In [5]:
# spark.sql(query).show(3)

In [6]:
# Warm up Q1

# query = '''
# SELECT 
#     COUNT(DISTINCT subreddit_id) count
# FROM
#    reddit_data
# '''

In [7]:
# spark.sql(query).show(3) 
#  78201 (reddit_201701)

In [8]:
df.head(3)

[Row(archived=True, author='halftone84', author_cakeday=None, author_flair_css_class=None, author_flair_text=None, body='I read the title and thought of that cheating bitch clown from the glassjaw video', body_html=None, controversiality=0, created=None, created_utc='1309478410', distinguished=None, downs=0, edited='false', gilded=0, id='c22x4bc', link_id='t3_idgji', mod_reports=None, name='t1_c22x4bc', parent_id='t3_idgji', removal_reason=None, replies=None, retrieved_on=1427302517, saved=None, score=1, score_hidden=False, stickied=None, subreddit='AskReddit', subreddit_id='t5_2qh1i', ups=1, user_reports=None),
 Row(archived=True, author='keiyakins', author_cakeday=None, author_flair_css_class=None, author_flair_text=None, body="Because you're about to wash your hands, but not your feet.", body_html=None, controversiality=0, created=None, created_utc='1309478412', distinguished=None, downs=0, edited='false', gilded=0, id='c22x4bi', link_id='t3_idez1', mod_reports=None, name='t1_c22x4b

In [9]:
df.columns

['archived',
 'author',
 'author_cakeday',
 'author_flair_css_class',
 'author_flair_text',
 'body',
 'body_html',
 'controversiality',
 'created',
 'created_utc',
 'distinguished',
 'downs',
 'edited',
 'gilded',
 'id',
 'link_id',
 'mod_reports',
 'name',
 'parent_id',
 'removal_reason',
 'replies',
 'retrieved_on',
 'saved',
 'score',
 'score_hidden',
 'stickied',
 'subreddit',
 'subreddit_id',
 'ups',
 'user_reports']

In [10]:
df.printSchema()

root
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- body_html: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created: long (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- mod_reports: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- removal_reason: string (nullable = true)
 |-- replies: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- saved: boolean (nullable = true)
 |--

In [11]:
query = '''
SELECT
  subreddit, 
  body
FROM
  reddit_data
WHERE
  body != "[deleted]"
'''
text_all = spark.sql(query)
text_all.show()

+-----------------+--------------------+
|        subreddit|                body|
+-----------------+--------------------+
|        AskReddit|I read the title ...|
|        AskReddit|Because you're ab...|
|         politics|All he had was a ...|
|        AskReddit|Flushing with you...|
|             IAmA|I grew up in Texa...|
|       California|I did this alread...|
|             pics|Ctrl-F: *drr* *dr...|
|       reddit.com|Damn. I was wrong...|
|        AskReddit|          Me too. :/|
|        AskReddit|I don't know, but...|
|           trance|How did you do that?|
|           google|I can haz google+...|
|             IAmA|Months?  That's a...|
|         sandiego|Is that the Reddi...|
|       googleplus|I got my invite, ...|
|       technology|This sounds bette...|
|        worldnews|Israel handled it...|
|DrawingFromWithin|Yeah, I'm ruler g...|
|           gaming|I just found out ...|
|       reddit.com|Hope you can't wa...|
+-----------------+--------------------+
only showing top

In [12]:
def f(row):
    return (row['subreddit'], row['body'].split(" "))


In [15]:
words = text_all.rdd.map(f).reduceByKey(lambda a, b: a + b)
words.take(10)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 163 in stage 5.0 failed 1 times, most recent failure: Lost task 163.0 in stage 5.0 (TID 4407, localhost, executor driver): java.io.IOException: No space left on device
	at sun.nio.ch.FileDispatcherImpl.write0(Native Method)
	at sun.nio.ch.FileDispatcherImpl.write(FileDispatcherImpl.java:60)
	at sun.nio.ch.IOUtil.writeFromNativeBuffer(IOUtil.java:93)
	at sun.nio.ch.IOUtil.write(IOUtil.java:51)
	at sun.nio.ch.FileChannelImpl.write(FileChannelImpl.java:211)
	at sun.nio.ch.FileChannelImpl.transferToTrustedChannel(FileChannelImpl.java:516)
	at sun.nio.ch.FileChannelImpl.transferTo(FileChannelImpl.java:612)
	at org.apache.spark.util.Utils$.copyFileStreamNIO(Utils.scala:389)
	at org.apache.spark.util.Utils.copyFileStreamNIO(Utils.scala)
	at org.apache.spark.shuffle.sort.UnsafeShuffleWriter.mergeSpillsWithTransferTo(UnsafeShuff

In [11]:
query = '''
SELECT 
    body
FROM
   reddit_data
'''
test = spark.sql(query)
test.show(20)

+--------------------+
|                body|
+--------------------+
|I read the title ...|
|Because you're ab...|
|All he had was a ...|
|Flushing with you...|
|I grew up in Texa...|
|I did this alread...|
|Ctrl-F: *drr* *dr...|
|Damn. I was wrong...|
|          Me too. :/|
|I don't know, but...|
|           [deleted]|
|How did you do that?|
|I can haz google+...|
|Months?  That's a...|
|Is that the Reddi...|
|I got my invite, ...|
|This sounds bette...|
|Israel handled it...|
|Yeah, I'm ruler g...|
|I just found out ...|
+--------------------+
only showing top 20 rows



In [12]:
# query = '''
# SELECT
#   subreddit, 
#   body
# FROM
#    reddit_data
# WHERE 
#   subreddit = 'reddit.com'
# '''
# test = spark.sql(query)
# test.show(10)

In [13]:
query = '''
SELECT
  DISTINCT subreddit
FROM
  reddit_data
WHERE
  body != "[deleted]"
'''
text_subreddits = spark.sql(query)
text_subreddits.show()

+---------------+
|      subreddit|
+---------------+
|          anime|
|         travel|
|     MensRights|
|   SaltLakeCity|
|           udel|
|       lacrosse|
|         AdPorn|
|        Amateur|
|     ArtHistory|
|   bachelorchef|
|           Utah|
|        jewelry|
|     QuotesPorn|
|  sunshinecoast|
|youtubecomments|
|    creepypasta|
|          HPMOR|
|          jrotc|
|     TheKilling|
|     WahoosTipi|
+---------------+
only showing top 20 rows



In [14]:
text_subreddits

DataFrame[subreddit: string]

In [15]:
subreddits = []
i = 0
for row in text_subreddits.rdd.collect():
    subreddit = row['subreddit']
#     print(subreddit)
    subreddits.append(subreddit)
    i += 1
#     if i == 20:
#         break

subreddits

['anime',
 'travel',
 'MensRights',
 'SaltLakeCity',
 'udel',
 'lacrosse',
 'AdPorn',
 'Amateur',
 'ArtHistory',
 'bachelorchef',
 'Utah',
 'jewelry',
 'QuotesPorn',
 'sunshinecoast',
 'youtubecomments',
 'creepypasta',
 'HPMOR',
 'jrotc',
 'TheKilling',
 'WahoosTipi',
 'freelanceWriters',
 'RecruitBigDanV',
 'Whippets',
 'couchsurfing',
 'costa_rica',
 'bookshelf',
 'BCCHigh',
 'electroswing',
 'charteroak',
 'NIU',
 'marvelheroes',
 'ASRoma',
 'ukraina',
 'TrueOffMyChest',
 'kitchener',
 'redditisfun',
 'mistyfront',
 'harasser',
 'MaddenMobile',
 'electrical',
 'UnresolvedMysteries',
 'Goldfish',
 'StLouisBeer',
 'DIYBeauty',
 'DRKCoin',
 'left4dead',
 'realitytv',
 'OnlyFoolsAndHorses',
 'Entrepreneurship',
 'Warcraft',
 'c0daisajerk',
 'ItsNotOnion',
 'UVU',
 'UAB',
 'washyourcigs',
 'HoJang',
 'Worcester',
 'Nightmares',
 'debunked',
 'lojban',
 'redditresort',
 'de4n',
 'htconex',
 'SCW',
 'metro2033',
 'MexicoIAMA',
 'DotCom',
 'Fairbanks',
 'ProjectRS06',
 'londonfootballmeetu

In [16]:
i = 0
words = []
for subreddit in subreddits:
    print(subreddit)
    
    query = '''
    SELECT
      subreddit, 
      body
    FROM
      reddit_data
    WHERE
      subreddit = "%s"
      AND
      body != "[deleted]"
    ''' % (subreddit)

    print("querying...")
    text_subreddit = spark.sql(query)
    
    words_subreddit = []
    for row in text_subreddit.rdd.collect():
        for word in row["body"].split(" "):
            words_subreddit.append(word)
    
    words.append(words_subreddit)
    
    i += 1
#     if i == 2:
#         break

words

anime
querying...


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 973 in stage 9.0 failed 1 times, most recent failure: Lost task 973.0 in stage 9.0 (TID 9324, localhost, executor driver): ExecutorLostFailure (executor driver exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 154814 ms
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1889)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1877)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1876)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGSch

In [18]:
words[1]

['Yeah,',
 "it's",
 'like',
 'saying',
 '"women',
 'are',
 'better',
 'at',
 'getting',
 'custody',
 'of',
 'their',
 'children',
 'in',
 'court"',
 'and',
 'bragging',
 'about',
 'it.',
 'You',
 'mean',
 'carcinogenic',
 'gasses?',
 '\n\nSorry',
 'for',
 'the',
 'pedantry,',
 'I',
 'do',
 'agree',
 'though.',
 "They're",
 'great',
 'jobs',
 'for',
 'young',
 'people.',
 'Not',
 'so',
 'much',
 'family',
 'people.',
 'A',
 'voice',
 'for',
 'men',
 'radio',
 'and',
 'www.register-her.com',
 'whats',
 'the',
 'context',
 'here?',
 'I',
 'think',
 'we',
 'all',
 'agree',
 'quotas',
 'are',
 'dumb,',
 'when',
 'they',
 'are',
 'brought',
 'up',
 '(outside)',
 'we',
 'should',
 'present',
 'something',
 'like',
 'a',
 '[blind',
 'audition](http://en.wikipedia.org/wiki/Blind_audition),',
 'where',
 'the',
 'potential',
 'for',
 'gender',
 'bias',
 'is',
 'removed',
 'or',
 'minimized.',
 "\n\nIt's",
 'going',
 'to',
 'be',
 'easier',
 'fight',
 'against',
 'this',
 'type',
 'of',
 'sexism',

In [ ]:
# Make dictionary which has indices and unique word. 
from gensim.corpora import Dictionary
dictionary = Dictionary(words)
# Make vectors. 
vectors = [dictionary.doc2bow(text) for text in words]
# TF-IDF
from gensim.models import TfidfModel
tfidf = TfidfModel(vectors)
# Use the first doc for word cloud. 
weights = tfidf[vectors[0]]
# Get terms from the dictionary and pair with weights. 
weights = [(dictionary[pair[0]], pair[1]) for pair in weights]
# Convert weights to dictionary. 
weights_dict = {}
for k, v in weights:
    weights_dict[k] = v

In [ ]:
# Initialize the word cloud. 

from wordcloud import WordCloud
word_cloud = WordCloud(
    background_color="white",
    max_words=2000,
    width = 1024,
    height = 720,
)

# Generate the cloud. 
word_cloud.generate_from_frequencies(weights_dict)
# Save the word cloud as a file. 
word_cloud.to_file("word_cloud.png")

In [13]:
query = '''
SELECT
  id, 
  body
FROM
  reddit_data
WHERE
  subreddit = "technology"
  AND
  body != "[deleted]"
'''
text_tech = spark.sql(query)
text_tech.show()

+-------+--------------------+
|     id|                body|
+-------+--------------------+
|dbumo2q|The one analogy I...|
|dbumoz0|&gt; peace, blue ...|
|dbumpim|All of a sudden, ...|
|dbumq8n|HTML5 was beginni...|
|dbumqf6|Turn off notifica...|
|dbumtjp|&gt; The mbp has ...|
|dbumtju|I think one thing...|
|dbumv4w|I'm including ord...|
|dbumxhj|Definitely looks ...|
|dbumy8v|Ah yes spec sheet...|
|dbumzys|admin work should...|
|dbun16b|Idk if you're bei...|
|dbun1cu|See my top post f...|
|dbun1o7|Hello! Unfortunat...|
|dbun1z1|Here in Belgium t...|
|dbun2ci|Don't let the hat...|
|dbun4g3|Quick question......|
|dbun5ek|&gt; Lets see som...|
|dbun5jx|I use mine to pay...|
|dbun6c7|Yeah, because fir...|
+-------+--------------------+
only showing top 20 rows



In [14]:
text_tech.select("body").show()

+--------------------+
|                body|
+--------------------+
|The one analogy I...|
|&gt; peace, blue ...|
|All of a sudden, ...|
|HTML5 was beginni...|
|Turn off notifica...|
|&gt; The mbp has ...|
|I think one thing...|
|I'm including ord...|
|Definitely looks ...|
|Ah yes spec sheet...|
|admin work should...|
|Idk if you're bei...|
|See my top post f...|
|Hello! Unfortunat...|
|Here in Belgium t...|
|Don't let the hat...|
|Quick question......|
|&gt; Lets see som...|
|I use mine to pay...|
|Yeah, because fir...|
+--------------------+
only showing top 20 rows



In [15]:
text_tech.head().body

'The one analogy I though of was that, back then, we were like satelites, or rockets: Only a handful of manufactures that could dictate the rules.\n\nSo it felt we were in control, forever and ever, but thats all we were, the only hot girl in the sausage party.\n\nNow a days were like the clothing store, if they can, they will shop online, but otherwise, in a hurry, or to someone that has some extra money will buy locally.'

In [16]:
text_tech

DataFrame[id: string, body: string]

In [17]:
text_tech.printSchema()

root
 |-- id: string (nullable = true)
 |-- body: string (nullable = true)



In [18]:
df = (text_tech
  .rdd
  .map(lambda x : (x.id, x.body.split(" ")))
  .toDF()
  .withColumnRenamed("_1","doc_id")
  .withColumnRenamed("_2","features"))

In [19]:
df.show()

+-------+--------------------+
| doc_id|            features|
+-------+--------------------+
|dbumo2q|[The, one, analog...|
|dbumoz0|[&gt;, peace,, bl...|
|dbumpim|[All, of, a, sudd...|
|dbumq8n|[HTML5, was, begi...|
|dbumqf6|[Turn, off, notif...|
|dbumtjp|[&gt;, The, mbp, ...|
|dbumtju|[I, think, one, t...|
|dbumv4w|[I'm, including, ...|
|dbumxhj|[Definitely, look...|
|dbumy8v|[Ah, yes, spec, s...|
|dbumzys|[admin, work, sho...|
|dbun16b|[Idk, if, you're,...|
|dbun1cu|[See, my, top, po...|
|dbun1o7|[Hello!, Unfortun...|
|dbun1z1|[Here, in, Belgiu...|
|dbun2ci|[Don't, let, the,...|
|dbun4g3|[Quick, question....|
|dbun5ek|[&gt;, Lets, see,...|
|dbun5jx|[I, use, mine, to...|
|dbun6c7|[Yeah,, because, ...|
+-------+--------------------+
only showing top 20 rows



In [20]:
words = [row["features"] for row in df.rdd.collect()]
# words[0]

In [21]:
# Make dictionary which has indices and unique word. 
from gensim.corpora import Dictionary
dictionary = Dictionary(words)

In [22]:
# Make vectors. 
vectors = [dictionary.doc2bow(text) for text in words]

In [23]:
#  TF-IDF
from gensim.models import TfidfModel
tfidf = TfidfModel(vectors)

In [24]:
# Use the first doc for word cloud. 
weights = tfidf[vectors[0]]

In [25]:
# Get terms from the dictionary and pair with weights. 
weights = [(dictionary[pair[0]], pair[1]) for pair in weights]

In [26]:
# Convert weights to dictionary. 
weights_dict = {}
for k, v in weights:
    weights_dict[k] = v

In [27]:
# Initialize the word cloud. 

from wordcloud import WordCloud
word_cloud = WordCloud(
    background_color="white",
    max_words=2000,
    width = 1024,
    height = 720,
)

# Generate the cloud. 
word_cloud.generate_from_frequencies(weights_dict)

In [28]:
# Save the word cloud as a file. 
word_cloud.to_file("word_cloud.png")

In [29]:
# from pyspark.ml.feature import HashingTF, IDF

# # TF
# htf = HashingTF(inputCol="features", outputCol="tf")
# tf = htf.transform(df)
# tf.show(truncate=False)
# # tf.show()

In [30]:
# # TF-IDF
# idf = IDF(inputCol="tf", outputCol="idf")
# tfidf = idf.fit(tf).transform(tf)
# tfidf.show(truncate=False)